# Interacting with Huggingface (HF)

### HF Limitations for our datasets
To upload our datasets to HF for easy distribution we use a non-trivial approach as most of our datasets are very large and HF has limits
- Not more than 10k files per folder
- Not more than 100k in total
- Loading with COCO format is not really supported
  
### Our approach
Because of this we zip all image and label files in the respective folders when uploading and unzip them when downloading. Just follow this setup as it gides you through the process. 

Note: Normal Git usage with HF is not really supported as tracking doesnt work since we zip/unzip

## Imports

In [1]:
import os
import zipfile
from huggingface_hub import HfApi, HfFolder
import shutil

/data22/stu236894/GitRepos/TinyML-MT/basket/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Login to HF
The cache is not directly used for our method but just in case

In [2]:
# HF Cache
os.environ["HF_HOME"] = "../../.cache"
!echo $HF_HOME
!huggingface-cli whoami

../../.cache


TorgeSchwark
orgs:  Basket-AEye


## Settings

In [3]:
DATASET_DIR = "ai_shelf" # Starting from a local huggingface folder in the repo
HF_REPO = "Basket-AEye/ai_shelf"

# Upload to HF

We only zip in subdirectories so any txt, json files in the first level are not zipped and can be used for HF settings and so on

In [11]:
created_zips = []

# ----- Functions -----
def zip_folder(folder_path, output_zip):
    """Zips the contents of a folder."""
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)
    print(f"Folder zipped to {output_zip}")
    created_zips.append(output_zip)

def upload_to_hf(path, repo_path):
    """Uploads a file to Hugging Face Hub."""
    api = HfApi()
    api.upload_file(
        path_or_fileobj=path,
        path_in_repo=repo_path,
        repo_id=HF_REPO,
        repo_type="dataset"
    )
    print(f"File {repo_path} uploaded to {HF_REPO}")

complete_dir = os.path.join("../../huggingface/", DATASET_DIR)

# Go through each folder in the dataset directory
def process_folder(folder_path, repo_prefix):
    """Processes a folder, zipping and uploading its contents recursively."""
    contains_image_or_text = any(
        file.endswith(('.png', '.jpg', '.jpeg', '.txt', '.json'))
        for file in os.listdir(folder_path)
    )
    if contains_image_or_text:
        zip_path = f"{folder_path}.zip"
        zip_folder(folder_path, zip_path)
        upload_to_hf(zip_path, repo_prefix + f"/{os.path.basename(folder_path)}.zip")
    else:
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            if os.path.isdir(item_path):
                process_folder(item_path, repo_prefix + f"/{os.path.basename(folder_path)}")
            elif os.path.isfile(item_path):
                upload_to_hf(item_path, repo_prefix + f"/{os.path.basename(folder_path)}/{item}")

# ----- Main -----
if not os.path.exists(complete_dir):
    print(f"Directory {complete_dir} does not exist.")
    exit(1)

# Get a list of all folders in the complete directory
folder_paths = [os.path.join(complete_dir, item) for item in os.listdir(complete_dir) if os.path.isdir(os.path.join(complete_dir, item))]

# Execute process_folder for each folder in the list
for folder_path in folder_paths:
    process_folder(folder_path, "")


Folder zipped to ../../huggingface/ai_shelf/sd/cereal.zip


No files have been modified since last commit. Skipping to prevent empty commit.


File /sd/cereal.zip uploaded to Basket-AEye/ai_shelf
Folder zipped to ../../huggingface/ai_shelf/sd/test/apple.zip


No files have been modified since last commit. Skipping to prevent empty commit.


File /sd/test/apple.zip uploaded to Basket-AEye/ai_shelf
Folder zipped to ../../huggingface/ai_shelf/sd/test/lemon.zip


...

File /sd/second/tomato sauce.zip uploaded to Basket-AEye/ai_shelf


No files have been modified since last commit. Skipping to prevent empty commit.


File /sd/second/fruit tea.zip uploaded to Basket-AEye/ai_shelf


No files have been modified since last commit. Skipping to prevent empty commit.


File /sd/cereal.zip uploaded to Basket-AEye/ai_shelf


### Remove zip files

In [12]:
for zip_file in created_zips:
    if os.path.exists(zip_file):
        os.remove(zip_file)
        print(f"Removed {zip_file}")
    else:
        print(f"{zip_file} does not exist.")
created_zips.clear()

Removed ../../huggingface/ai_shelf/sd/cereal.zip
Removed ../../huggingface/ai_shelf/sd/test/apple.zip
Removed ../../huggingface/ai_shelf/sd/test/lemon.zip
Removed ../../huggingface/ai_shelf/sd/test/banana.zip
Removed ../../huggingface/ai_shelf/sd/test/cucumber.zip
Removed ../../huggingface/ai_shelf/sd/test/avocado.zip
Removed ../../huggingface/ai_shelf/sd/test/tomato sauce.zip
Removed ../../huggingface/ai_shelf/sd/test/fruit tea.zip
Removed ../../huggingface/ai_shelf/sd/test/coffee.zip
Removed ../../huggingface/ai_shelf/sd/test/spaghetti.zip
Removed ../../huggingface/ai_shelf/sd/test/coke.zip
Removed ../../huggingface/ai_shelf/sd/first/tomato_sauce_.zip
Removed ../../huggingface/ai_shelf/sd/first/fruit_tea_.zip
Removed ../../huggingface/ai_shelf/sd/first/coffee_.zip
Removed ../../huggingface/ai_shelf/sd/first/spaghetti_.zip
Removed ../../huggingface/ai_shelf/sd/first/coke.zip
Removed ../../huggingface/ai_shelf/sd/first/apple.zip
Removed ../../huggingface/ai_shelf/sd/first/lemon.zip
Rem

# Download from HF

This process removes the local directory, redownloads the entire repo and unzips

In [4]:
from huggingface_hub import snapshot_download

complete_dir = os.path.join("../../huggingface/", DATASET_DIR)

# Check if the path exists and delete it
if os.path.exists(complete_dir):
    #shutil.rmtree(complete_dir)
    print(f"Existing directory: {complete_dir}")

# Download the repo from HF using snapshot_download
snapshot_path = snapshot_download(
    repo_id=HF_REPO,
    repo_type="dataset",
    local_dir=complete_dir
)
print(f"Downloaded snapshot of repo {HF_REPO} to {snapshot_path}")
# Unzip all zip files recursively and put files into a folder with the name of the zip
for root, dirs, files in os.walk(snapshot_path):
    for file in files:
        if file.endswith('.zip'):
            zip_path = os.path.join(root, file)
            folder_name = os.path.splitext(file)[0]  # Get the name of the zip file without extension
            extract_path = os.path.join(root, folder_name)  # Create a folder with the name of the zip
            os.makedirs(extract_path, exist_ok=True)  # Ensure the folder exists
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)  # Extract files into the folder
            # os.remove(zip_path)  # Uncomment for better caching but takes more space
            print(f"Unzipped {zip_path} into {extract_path}")


Fetching 29 files: 100%|██████████| 29/29 [00:04<00:00,  6.19it/s]


Downloaded snapshot of repo Basket-AEye/ai_shelf to /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf
Unzipped /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/cereal.zip into /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/cereal
Unzipped /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/banana.zip into /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/banana
Unzipped /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/cucumber.zip into /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/cucumber
Unzipped /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/avocado.zip into /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/avocado
Unzipped /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/coke.zip into /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/first/coke
Unzipped /data22/stu236894/GitRepos/TinyML-MT/huggingface/ai_shelf/sd/

## Remove zip files (No caching)
When downloading we leave the zip files for caching: Comparing the two zip files and then not redownloading if same (TAKES MORE STORAGE THOUGH)
If not wanted execute the cell below

In [5]:
complete_dir = os.path.join("../../huggingface/", DATASET_DIR)

for root, dirs, files in os.walk(complete_dir):
    for file in files:
        if file.endswith('.zip'):
            zip_path = os.path.join(root, file)
            os.remove(zip_path)
            print(f"Removed {zip_path}")

Removed ../../huggingface/ai_shelf/sd/cereal.zip
Removed ../../huggingface/ai_shelf/sd/first/banana.zip
Removed ../../huggingface/ai_shelf/sd/first/cucumber.zip
Removed ../../huggingface/ai_shelf/sd/first/avocado.zip
Removed ../../huggingface/ai_shelf/sd/first/coke.zip
Removed ../../huggingface/ai_shelf/sd/first/apple.zip
Removed ../../huggingface/ai_shelf/sd/first/lemon.zip
Removed ../../huggingface/ai_shelf/sd/first/tomato_sauce_.zip
Removed ../../huggingface/ai_shelf/sd/first/coffee_.zip
Removed ../../huggingface/ai_shelf/sd/first/fruit_tea_.zip
Removed ../../huggingface/ai_shelf/sd/first/spaghetti_.zip
Removed ../../huggingface/ai_shelf/sd/second/apple.zip
Removed ../../huggingface/ai_shelf/sd/second/fruit tea.zip
Removed ../../huggingface/ai_shelf/sd/second/cucumber.zip
Removed ../../huggingface/ai_shelf/sd/second/tomato sauce.zip
Removed ../../huggingface/ai_shelf/sd/second/lemon.zip
Removed ../../huggingface/ai_shelf/sd/second/banana.zip
Removed ../../huggingface/ai_shelf/sd/sec